In [2]:
import os
import shutil
import xml.etree.ElementTree as ET

# 폴더 경로 설정
xml_dir = "./XML"
images_dir = "./images"
txt_dir = "./TXT"
txt_class_dir = "./TXT_class"
convert_dir = "./CONVERT"
images_class_dir = "./images_class"

# 필요한 폴더 생성
os.makedirs(txt_dir, exist_ok=True)
os.makedirs(txt_class_dir, exist_ok=True)
os.makedirs(images_class_dir, exist_ok=True)
os.makedirs(convert_dir, exist_ok=True)

# 분류 결과를 저장할 파일 경로 설정
class0_path = os.path.join(convert_dir, "class0.txt")

# 사용자 입력을 통해 class0으로 사용할 클래스 이름 설정
class0_name = input("class0으로 사용할 클래스 이름을 입력하세요 (예: D40): ")

# class0 결과를 저장할 리스트
class0_files = []

# XML 파일을 YOLO 형식으로 변환하는 함수 (D40만 유지)
def xml_to_yolo(xml_path, output_txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_width = int(root.find('size/width').text)
    image_height = int(root.find('size/height').text)

    yolo_annotations = []
    has_class0 = False

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name == class0_name:  # 지정된 클래스(D40)만 처리
            has_class0 = True
            class_id = 0

            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / image_width
            y_center = ((ymin + ymax) / 2) / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    if has_class0:  # D40이 포함된 경우에만 YOLO 텍스트 파일 생성
        with open(output_txt_path, "w") as file:
            for line in yolo_annotations:
                file.write(line + "\n")
        class0_files.append(output_txt_path)

# 파일 매칭 및 변환 처리
xml_files = sorted([f for f in os.listdir(xml_dir) if f.endswith(".xml")])
image_files = sorted([f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']])

xml_names = {os.path.splitext(f)[0] for f in xml_files}
image_names = {os.path.splitext(f)[0] for f in image_files}

# 매칭된 파일 확인
matched_files = xml_names.intersection(image_names)

# 매칭된 파일 변환 및 분류
for matched in matched_files:
    xml_path = os.path.join(xml_dir, matched + ".xml")
    txt_filename = matched + ".txt"
    output_txt_path = os.path.join(txt_dir, txt_filename)

    # XML 파일을 YOLO 형식으로 변환하여 TXT 폴더에 저장 (D40만 유지)
    xml_to_yolo(xml_path, output_txt_path)

# D40만 포함된 파일만 TXT_class와 images_class로 이동
for file_path in class0_files:
    filename = os.path.basename(file_path)
    txt_class_output_path = os.path.join(txt_class_dir, filename)

    # TXT 파일을 TXT_class 폴더로 이동
    shutil.move(file_path, txt_class_output_path)

    # 해당하는 이미지 파일을 images_class 폴더로 복사
    image_name = os.path.splitext(filename)[0]
    for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']:
        image_path = os.path.join(images_dir, image_name + ext)
        if os.path.exists(image_path):
            shutil.copy(image_path, images_class_dir)
            break  # 이미지를 찾고 복사했으면 종료

# 결과 파일 기록
with open(class0_path, "w") as f0:
    for file in class0_files:
        f0.write(file + "\n")

# 처리 결과 출력
print(f"class0.txt에 {len(class0_files)}개의 파일이 기록되었습니다.")
print(f"TXT_class와 images_class 폴더에 D40 클래스만 포함된 파일이 저장되었습니다.")


class0.txt에 164개의 파일이 기록되었습니다.
TXT_class와 images_class 폴더에 D40 클래스만 포함된 파일이 저장되었습니다.
